# 📊 Negotiation Session Analysis Dashboard

This notebook provides comprehensive analysis of negotiation sessions from the database.

## Features:
- View all negotiation sessions with key information
- Filter by configuration (Base, M, M+P)
- Track Memory (M) and Planning (P) evolution across rounds
- Analyze deal outcomes and performance

In [1]:
# Import required libraries
import sqlite3
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

print("✅ Libraries loaded successfully")

✅ Libraries loaded successfully


## 1. Database Connection

In [32]:
# Connect to database
DB_PATH = 'negotiations (2).db'  # Update this path if needed

try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Get total number of sessions
    cursor.execute("SELECT COUNT(*) FROM negotiation_sessions")
    total_sessions = cursor.fetchone()[0]
    
    print(f"✅ Connected to database: {DB_PATH}")
    print(f"📊 Total sessions in database: {total_sessions}")
except Exception as e:
    print(f"❌ Error connecting to database: {e}")
    print("Please update DB_PATH variable with the correct database location")

✅ Connected to database: negotiations (2).db
📊 Total sessions in database: 113


## 2. View All Sessions (Summary)

In [33]:
# Load all sessions with key information
query = """
SELECT 
    session_id,
    scenario_name,
    student_role,
    ai_model,
    CASE 
        WHEN student_goes_first = 1 THEN '👤 Student'
        ELSE '🤖 AI'
    END as first_mover,
    CASE 
        WHEN use_memory = 1 AND use_plan = 1 THEN '🧠🎯 M+P'
        WHEN use_memory = 1 THEN '🧠 M'
        ELSE '⚪ Base'
    END as config,
    total_rounds,
    CASE 
        WHEN deal_reached = 1 THEN '✅ Deal'
        WHEN deal_failed = 1 THEN '❌ Failed'
        WHEN status = 'completed' THEN '⏹️ No Deal'
        ELSE '🔄 ' || status
    END as outcome,
    major,
    gender,
    negotiation_experience,
    created_at,
    updated_at
FROM negotiation_sessions
ORDER BY created_at DESC
"""

df_sessions = pd.read_sql_query(query, conn)

# Format timestamps
df_sessions['created_at'] = pd.to_datetime(df_sessions['created_at']).dt.strftime('%Y-%m-%d %H:%M')
df_sessions['updated_at'] = pd.to_datetime(df_sessions['updated_at']).dt.strftime('%Y-%m-%d %H:%M')

# Display
print(f"\n📋 Total Sessions: {len(df_sessions)}\n")
display(df_sessions)


📋 Total Sessions: 113



,session_id,scenario_name,student_role,ai_model,first_mover,config,total_rounds,outcome,major,gender,negotiation_experience,created_at,updated_at
0,cba42d83-4de7-41aa-9c2d-fb6d6378355e,Main_Street,side2,openai/gpt-5,🤖 AI,🧠🎯 M+P,6,🔄 active,SDS,,,2025-12-10 15:28,2025-12-10 15:29
1,c18ba8b7-2605-441f-aa02-e9c0fc8cf80c,Main_Street,side2,openai/gpt-5,🤖 AI,⚪ Base,6,🔄 active,SDS,,,2025-12-10 15:28,2025-12-10 15:29
2,21380327-cdb4-4f7f-873b-cb1e6c012017,Main_Street,side2,openai/gpt-5,👤 Student,⚪ Base,6,🔄 active,SDS,,,2025-12-10 15:28,2025-12-10 15:28
3,44902306-321c-4faf-8add-e73451889ad5,Main_Street,side2,openai/gpt-5,🤖 AI,⚪ Base,6,🔄 active,Business,,,2025-12-10 15:28,2025-12-10 15:28
4,9b40af5c-bf64-483e-bb6b-e60855980604,Main_Street,side2,openai/gpt-5,👤 Student,🧠🎯 M+P,6,🔄 active,SDS,,,2025-12-10 15:28,2025-12-10 15:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,7271f344-12b8-46cf-8892-f69ebec7dde3,Main_Street,side2,openai/gpt-5,👤 Student,⚪ Base,6,✅ Deal,SDS,,,2025-12-10 00:11,2025-12-10 00:16
109,362cd512-ee83-4ed6-bf80-8d5ea8ce6f49,Top_talent,side2,openai/gpt-5,👤 Student,⚪ Base,6,🔄 failed,SDS,,,2025-12-10 00:09,2025-12-10 00:16
110,20b2f876-42ae-4be1-a9b5-a9347f3b50af,Top_talent,side2,openai/gpt-5,👤 Student,🧠🎯 M+P,6,🔄 active,SDS,,,2025-12-10 00:06,2025-12-10 00:06
111,67f91417-c064-44ff-b0e9-65404a64f8df,Main_Street,side2,openai/gpt-5,🤖 AI,🧠🎯 M+P,6,🔄 active,SDS,Male,Occasional,2025-12-10 00:05,2025-12-10 00:06


## 3. Configuration Summary Statistics

In [34]:
# Configuration breakdown
print("\n📊 Configuration Breakdown:\n")

config_stats = df_sessions.groupby('config').agg({
    'session_id': 'count',
    'outcome': lambda x: (x.str.contains('Deal')).sum()
}).rename(columns={'session_id': 'Total', 'outcome': 'Deals'})

config_stats['Deal Rate'] = (config_stats['Deals'] / config_stats['Total'] * 100).round(1).astype(str) + '%'

display(config_stats)

# First mover breakdown
print("\n👥 First Mover Breakdown:\n")
display(df_sessions['first_mover'].value_counts())

# Scenario breakdown
print("\n🎭 Scenario Breakdown:\n")
display(df_sessions['scenario_name'].value_counts())


📊 Configuration Breakdown:



,Total,Deals,Deal Rate
config,,,
⚪ Base,63,27,42.9%
🧠🎯 M+P,50,25,50.0%



👥 First Mover Breakdown:



first_mover
🤖 AI         59
👤 Student    54
Name: count, dtype: int64


🎭 Scenario Breakdown:



scenario_name
Main_Street    107
Top_talent       6
Name: count, dtype: int64

## 4. Select a Session to Analyze

In [35]:
# Display sessions for selection
print("\n📌 Available Sessions (most recent 20):\n")

recent_sessions = df_sessions.head(20)[['session_id', 'scenario_name', 'config', 'first_mover', 'outcome', 'created_at']]
display(recent_sessions)

print("\n⬇️ Copy a session_id from above and paste it below:")


📌 Available Sessions (most recent 20):



,session_id,scenario_name,config,first_mover,outcome,created_at
0,cba42d83-4de7-41aa-9c2d-fb6d6378355e,Main_Street,🧠🎯 M+P,🤖 AI,🔄 active,2025-12-10 15:28
1,c18ba8b7-2605-441f-aa02-e9c0fc8cf80c,Main_Street,⚪ Base,🤖 AI,🔄 active,2025-12-10 15:28
2,21380327-cdb4-4f7f-873b-cb1e6c012017,Main_Street,⚪ Base,👤 Student,🔄 active,2025-12-10 15:28
3,44902306-321c-4faf-8add-e73451889ad5,Main_Street,⚪ Base,🤖 AI,🔄 active,2025-12-10 15:28
4,9b40af5c-bf64-483e-bb6b-e60855980604,Main_Street,🧠🎯 M+P,👤 Student,🔄 active,2025-12-10 15:28
5,519ed03e-cbf5-4ac8-b77a-1e4ee1b7d01f,Main_Street,⚪ Base,👤 Student,🔄 active,2025-12-10 15:25
6,d61b1421-2419-4a8a-832c-f426db673ebe,Main_Street,⚪ Base,👤 Student,🔄 active,2025-12-10 15:10
7,8f327691-8c64-4eb3-913a-a68d55ae1737,Main_Street,⚪ Base,🤖 AI,✅ Deal,2025-12-10 15:09
8,49f9367b-bdb8-44cf-b6d2-683f825e5e96,Main_Street,⚪ Base,🤖 AI,🔄 active,2025-12-10 15:09
9,def6ce77-9078-42e8-989d-7ec7e9135680,Main_Street,⚪ Base,🤖 AI,✅ Deal,2025-12-10 15:07



⬇️ Copy a session_id from above and paste it below:


In [10]:
# 👇 PASTE SESSION ID HERE
selected_session_id = "d1c19ae5-07a8-4f1c-af38-7c50758e9c32"

# Validate and load session
if selected_session_id == "d1c19ae5-07a8-4f1c-af38-7c50758e9c32":
    print("⚠️ Please update 'selected_session_id' with an actual session ID from the table above")
else:
    query = f"SELECT * FROM negotiation_sessions WHERE session_id = '{selected_session_id}'"
    session = pd.read_sql_query(query, conn)
    
    if len(session) == 0:
        print(f"❌ Session ID '{selected_session_id}' not found")
    else:
        print(f"✅ Loaded session: {selected_session_id}")
        print(f"   Scenario: {session.iloc[0]['scenario_name']}")
        print(f"   Config: {'M+P' if session.iloc[0]['use_memory'] and session.iloc[0]['use_plan'] else 'M' if session.iloc[0]['use_memory'] else 'Base'}")
        print(f"   First Mover: {'Student' if session.iloc[0]['student_goes_first'] else 'AI'}")

⚠️ Please update 'selected_session_id' with an actual session ID from the table above


## 5. View Transcript (Conversation)

In [18]:
if selected_session_id != "d1c19ae5-07a8-4f1c-af38-7c50758e9c32" and len(session) > 0:
    print("\n💬 Negotiation Transcript:\n")
    print("="*100)
    
    transcript = json.loads(session.iloc[0]['transcript'])
    
    for i, message in enumerate(transcript, 1):
        print(f"\n[{i}] {message}")
        print("-"*100)
    
    print(f"\n📊 Total messages: {len(transcript)}")

## 6. Track Memory (M) Evolution

In [19]:
if selected_session_id != "d1c19ae5-07a8-4f1c-af38-7c50758e9c32" and len(session) > 0:
    if session.iloc[0]['use_memory']:
        print("\n🧠 Memory Module Evolution Across Rounds:\n")
        print("="*120)
        
        memory_history = json.loads(session.iloc[0]['ai_memory_history'])
        
        if len(memory_history) == 0:
            print("⚠️ No memory history recorded (session may have ended early)")
        else:
            for i, memory_state in enumerate(memory_history, 1):
                print(f"\n{'='*120}")
                print(f"📍 ROUND {i} - Memory State")
                print(f"{'='*120}")
                
                # Parse memory state
                if isinstance(memory_state, str):
                    memory_state = json.loads(memory_state)
                
                # Display each memory category
                for category, content in memory_state.items():
                    print(f"\n🔹 {category.upper()}:")
                    if isinstance(content, list):
                        for item in content:
                            print(f"   • {item}")
                    else:
                        print(f"   {content}")
            
            print(f"\n\n📊 Summary: {len(memory_history)} memory states tracked across rounds")
    else:
        print("\n⚠️ This session used Base configuration (no Memory module)")

## 7. Track Planning (P) Evolution

In [20]:
if selected_session_id != "d1c19ae5-07a8-4f1c-af38-7c50758e9c32" and len(session) > 0:
    if session.iloc[0]['use_plan']:
        print("\n🎯 Planning Module Evolution Across Rounds:\n")
        print("="*120)
        
        plan_history = json.loads(session.iloc[0]['ai_plan_history'])
        
        if len(plan_history) == 0:
            print("⚠️ No planning history recorded (session may have ended early)")
        else:
            for i, plan_state in enumerate(plan_history, 1):
                print(f"\n{'='*120}")
                print(f"📍 ROUND {i} - Planning State")
                print(f"{'='*120}\n")
                
                # Parse plan state
                if isinstance(plan_state, str):
                    plan_state = json.loads(plan_state)
                
                # Display planning sections
                for section, content in plan_state.items():
                    print(f"\n🔸 {section.upper().replace('_', ' ')}:")
                    if isinstance(content, dict):
                        for key, value in content.items():
                            print(f"   • {key}: {value}")
                    elif isinstance(content, list):
                        for item in content:
                            print(f"   • {item}")
                    else:
                        print(f"   {content}")
            
            print(f"\n\n📊 Summary: {len(plan_history)} planning states tracked across rounds")
    else:
        print("\n⚠️ This session did not use Planning module (Base or M only)")

## 8. Deal Analysis (if applicable)

In [ ]:
if selected_session_id != "YOUR_SESSION_ID_HERE" and len(session) > 0:
    if session.iloc[0]['deal_reached']:
        print("\n✅ DEAL REACHED\n")
        print("="*100)
        
        # Student's deal
        print("\n👤 Student's Deal Terms:")
        if session.iloc[0]['student_deal_json']:
            student_deal = json.loads(session.iloc[0]['student_deal_json'])
            for key, value in student_deal.items():
                print(f"   • {key}: {value}")
        else:
            print("   (No deal JSON recorded)")
        
        # AI's deal
        print("\n🤖 AI's Deal Terms:")
        if session.iloc[0]['ai_deal_json']:
            ai_deal = json.loads(session.iloc[0]['ai_deal_json'])
            for key, value in ai_deal.items():
                print(f"   • {key}: {value}")
        else:
            print("   (No deal JSON recorded)")
    
    elif session.iloc[0]['deal_failed']:
        print("\n❌ DEAL FAILED")
        print("\nReason: Fundamental constraint gap or verification failure")
    
    else:
        print("\n⏹️ NO DEAL REACHED")
        print("\nNegotiation ended without agreement (parties walk away with BATNA)")

## 9. Demographics Summary

In [17]:
if selected_session_id != "d1c19ae5-07a8-4f1c-af38-7c50758e9c32" and len(session) > 0:
    print("\n👥 Student Demographics:\n")
    print("="*50)
    print(f"   Major: {session.iloc[0]['major'] or 'Not provided'}")
    print(f"   Gender: {session.iloc[0]['gender'] or 'Not provided'}")
    print(f"   Negotiation Experience: {session.iloc[0]['negotiation_experience'] or 'Not provided'}")

## 10. Export Session Data

In [ ]:
# Export options
if selected_session_id != "YOUR_SESSION_ID_HERE" and len(session) > 0:
    
    # Option 1: Export full session to JSON
    export_data = {
        'session_id': session.iloc[0]['session_id'],
        'scenario': session.iloc[0]['scenario_name'],
        'configuration': {
            'use_memory': bool(session.iloc[0]['use_memory']),
            'use_plan': bool(session.iloc[0]['use_plan']),
            'first_mover': 'student' if session.iloc[0]['student_goes_first'] else 'ai'
        },
        'transcript': json.loads(session.iloc[0]['transcript']),
        'memory_history': json.loads(session.iloc[0]['ai_memory_history']) if session.iloc[0]['use_memory'] else None,
        'plan_history': json.loads(session.iloc[0]['ai_plan_history']) if session.iloc[0]['use_plan'] else None,
        'outcome': {
            'deal_reached': bool(session.iloc[0]['deal_reached']),
            'deal_failed': bool(session.iloc[0]['deal_failed']),
            'student_deal': json.loads(session.iloc[0]['student_deal_json']) if session.iloc[0]['student_deal_json'] else None,
            'ai_deal': json.loads(session.iloc[0]['ai_deal_json']) if session.iloc[0]['ai_deal_json'] else None
        }
    }
    
    filename = f"session_{selected_session_id[:8]}.json"
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"\n💾 Session data exported to: {filename}")

## 11. Comparative Analysis (Multiple Sessions)

In [ ]:
# Compare sessions by configuration
print("\n📊 Comparative Analysis by Configuration:\n")

comparison = df_sessions.groupby('config').agg({
    'session_id': 'count',
    'total_rounds': 'mean',
    'outcome': lambda x: (x.str.contains('Deal')).sum()
}).rename(columns={
    'session_id': 'Total Sessions',
    'total_rounds': 'Avg Rounds',
    'outcome': 'Deals Reached'
})

comparison['Avg Rounds'] = comparison['Avg Rounds'].round(2)
comparison['Deal Rate %'] = (comparison['Deals Reached'] / comparison['Total Sessions'] * 100).round(1)

display(comparison)

## 12. Filter Sessions

In [ ]:
# Example: Filter for M+P configuration with deals
print("\n🔍 Example Filter: M+P Configuration with Deals Reached\n")

filtered = df_sessions[
    (df_sessions['config'].str.contains('M\+P')) & 
    (df_sessions['outcome'].str.contains('Deal'))
]

print(f"Found {len(filtered)} sessions matching criteria:\n")
display(filtered[['session_id', 'scenario_name', 'first_mover', 'outcome', 'created_at']])

# You can modify the filter conditions above to analyze different subsets

In [ ]:
# Close database connection
conn.close()
print("\n✅ Analysis complete. Database connection closed.")